In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Juninho\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Создадим простой граф
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
# Прогоним его
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
# Чтобы не повторять каждый раз название сессии
with tf.Session() as sess:    # Внутри блока with выбранный сеанс устанавливается в качестве стандартного
    x.initializer.run()       # Аналогично tf.get_default_session().run(x.initializer) 
    y.initializer.run()       # и т.д.
    result = f.eval()
    print(result)

42


In [5]:
# Вместо запуска инициализатора для каждой переменной вручную можно применить
init = tf.global_variables_initializer()  # только подготовка инициализирующего узла

with tf.Session() as sess:
    init.run()                            # инициализация переменных
    result1 = f.eval()

print(result1)

42


In [6]:
# можно также использовать InteractiveSession, который после создания делает себя стандартным сеансом
sess = tf.InteractiveSession()
init.run()         # и блок with не нужен

In [7]:
result = f.eval()    
print(result)
sess.close()       # но ее необходимо закрывать вручную

42


#### Управление графами

In [8]:
# любой созданный автоматически узел добавляется к стандартному графу
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
# Но если есть необходимость в использовании нескольких графов
graph = tf.Graph()          # Создав новый граф
with graph.as_default():    # Можно временно сделать новый граф стандартным
    x2 = tf.Variable(2)

In [10]:
# и тогда эта переменная входит в наш новый граф
x2.graph is graph

True

In [11]:
# и не входит в стандартный
x2.graph is tf.get_default_graph()

False

#### Жизненный цикл значения узла

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [13]:
# При прогоне такого графа следующим образом:
with tf.Session() as sess:
    print(y.eval())  # тут tensorflow определяет перемнные от которых зависит y(x, w) и оценивает их
    print(z.eval())  # НО ЗДЕСЬ ОН ДЕЛАЕТ тоже самое и заново расчитывает (x, w)

10
15


In [14]:
# Между прогонами графа значения всех узлов отбрасываются кроме значения переменных 
# чтобы оценить z и y не оценивая w и x дважды,
# необходимо запросить оценку z и y только в одном прогоне графа:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


#### Линейная регрессия с помощью tensorflow

In [15]:
import os
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [16]:
path = os.path.join(os.getcwd(), 'Data')
house_path = os.path.join(path, 'housing.csv')
house_path

'C:\\Users\\Juninho\\MLProjects\\Study\\Aurelien Geron - Hands-on ML with Tensorflow and Scikit-learn\\Tensorflow\\Tensorflow\\Data\\housing.csv'

In [17]:
housing = fetch_california_housing(data_home=path)

In [18]:
pd.DataFrame(housing.data, columns=housing.feature_names).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [19]:
# housing = pd.read_csv(house_path)
# housing.head()

In [20]:
# housing.info()  # total_bedrooms имеютпустые значения, которые надо заполнить

In [21]:
# from sklearn.preprocessing import OneHotEncoder

In [22]:
# # обработаем категориальный признак

# # метод factorize() возвращает np-массив с определенными числами для каждой категории и список категорий
# house_ocean, house_ocean_cat = housing['ocean_proximity'].factorize() 

# # Но для машинного обучения правильно будет развернуть в матрицу столбик на категорию
# # с единичным значением на категорию 
# encoder = OneHotEncoder()
# house_ocn_1hot_sprs = encoder.fit_transform(house_ocean.reshape(-1, 1))
# house_ocean_1hot = house_ocn_1hot_sprs.toarray()

In [23]:
# housing_target = housing['median_house_value'].copy()       # Выделим цель
# housing_d = housing.drop(['median_house_value', 'ocean_proximity'], axis=1) # и данные(пока без категориального признака)

# # найдем медиану по total_bedrooms 
# median = housing_d['total_bedrooms'].median()
# housing_d['total_bedrooms'].fillna(median, inplace=True)  # и заполним этой медианой пустые значения в total_bedrooms


# housing_d_cat = housing_d.columns  # Выделим названия столбцов для наших данных

# housing_data = np.concatenate([housing_d, house_ocean_1hot], axis=1) # объединим наши данные с категориальным признаком

# # добавляем столбики категорий в список наших столбцов
# housing_data_cat = np.hstack([housing_d_cat, house_ocean_cat])          

In [24]:
# housing_target = np.array(housing_target)   # приведем целевой массив к numpy-форме

In [25]:
# Просто загрузим данные
housing = fetch_california_housing(data_home=path)

In [26]:
pd.DataFrame(housing.data, columns=housing.feature_names).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [27]:
housing_d = housing.data
housing_target = housing.target

**Реализуем линейную регрессию с помощью tensorflow**

In [28]:
m, n = housing_d.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing_d]   # объединяет по slice по второму измерению

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X') 
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # вычисление с помощью нормального уравнения

with tf.Session() as sess:
    theta_value = theta.eval()

In [29]:
theta_value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

### Вычислим линейную регрессию с помощью градиентноого спуска

Изначально вычислим градиенты вручную по формуле частной производной функции издержек для линейной регрессии.

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
scaler = StandardScaler()

scaled_housing_data = scaler.fit_transform(housing_d)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [32]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [33]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [34]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Эпоха', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Эпоха 0 MSE= 9.161543
Эпоха 100 MSE= 0.7145006
Эпоха 200 MSE= 0.56670463
Эпоха 300 MSE= 0.5555716
Эпоха 400 MSE= 0.5488116
Эпоха 500 MSE= 0.5436362
Эпоха 600 MSE= 0.53962916
Эпоха 700 MSE= 0.5365092
Эпоха 800 MSE= 0.53406775
Эпоха 900 MSE= 0.53214705


In [35]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770885],
       [ 0.3617837 ],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.63752776]], dtype=float32)

### Использование autodiff

Предыдущий код требует математического выведения градиентов из функции издержек MSE. Для линейной регрессии это легко, но, например, для глубоких нейронных сетей утомительно и подвержено ошибкам. К счастью на выручку приходит средство **autodiff** библиотеки tensorflow, которое автоматически вычисляет градиенты. 

Для его применения достаточно просто в предыдущем коде **заменить строку gradients = ..**

In [36]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# выбираем первый элемент списка 
# так как theta единственная переменная для которой создается операция - в данном случае, то и в списке один элемент
gradients = tf.gradients(mse, [theta])[0]    

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Эпоха', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Эпоха 0 MSE= 9.161543
Эпоха 100 MSE= 0.7145006
Эпоха 200 MSE= 0.56670463
Эпоха 300 MSE= 0.5555716
Эпоха 400 MSE= 0.5488117
Эпоха 500 MSE= 0.5436362
Эпоха 600 MSE= 0.53962916
Эпоха 700 MSE= 0.53650916
Эпоха 800 MSE= 0.5340678
Эпоха 900 MSE= 0.53214717


Функция gradients() берет операцию (в данном случае mse) и список переменных(здесь только theta) и создает список операций(по одной на переменную) для вычисления градиентов операции относительно каждой переменной. Таким образом. узел gradients будет вычислять вектор-градиент MSE по отношению к theta.

Существует четыре основных подхода к автоматическому вычислению градиентов, Tensorflow использует *автоматическое дифференцирование в обратном режиме(reverse-mode autodiff)*, которое идеально в плане эффективности и правильности, когда есть много входных и мало выходных данных, как часто происходит в нейронных сетях. Оно вычисляет частные производные выходов всего за $n_{выходов} + 1$ обходов графа.

### Использование оптимизатора

Tensorflow предоставляет также несколько готовых оптимизаторов, включая оптимизатор градиентного спуска. 

Заменим gradients = .. и training_op = .. 

In [37]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# на такие строки и код снова успешно отработает
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)   # 
training_op = optimizer.minimize(mse)                                        #
##############################################################################

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Эпоха', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Эпоха 0 MSE= 9.161543
Эпоха 100 MSE= 0.7145006
Эпоха 200 MSE= 0.56670463
Эпоха 300 MSE= 0.5555716
Эпоха 400 MSE= 0.5488117
Эпоха 500 MSE= 0.5436362
Эпоха 600 MSE= 0.53962916
Эпоха 700 MSE= 0.53650916
Эпоха 800 MSE= 0.5340678
Эпоха 900 MSE= 0.53214717


При желании можно применить оптимизатор другого типа. Например *моментный оптимизатор(momentum optimizer)*, который часто сходится намного быстрее.

In [38]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)     ########

training_op = optimizer.minimize(mse)                                        


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Эпоха', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Эпоха 0 MSE= 9.161543
Эпоха 100 MSE= 0.53056407
Эпоха 200 MSE= 0.52501106
Эпоха 300 MSE= 0.52441096
Эпоха 400 MSE= 0.52433294
Эпоха 500 MSE= 0.5243222
Эпоха 600 MSE= 0.5243206
Эпоха 700 MSE= 0.52432084
Эпоха 800 MSE= 0.52432084
Эпоха 900 MSE= 0.5243207


### Передача данных алгоритму обучения

Реализуем мини-пакетный градиентный спуск в предыдущем алгоритме регрессии. Для этого нам необходим способ замены X и y на каждой итерации. Простой способ использовать узел-заполнитель(placeholder node). Для его создания используется функция placeholder() и указать тип данных выходного тензора. Например:

In [39]:
A = tf.placeholder(tf.float32, shape=(None, 3)) # Указание None для измерения означает "любой размер"
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [40]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')            # Заменим константы
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)   # 
training_op = optimizer.minimize(mse)                                         


init = tf.global_variables_initializer()

In [41]:
n_epochs = 10

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing_target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        for batch_index in range(batch_size):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run([training_op, mse], feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [42]:
best_theta

array([[ 2.0775304e+00],
       [ 8.9261961e-01],
       [ 1.4729246e-01],
       [-3.5954681e-01],
       [ 3.6123931e-01],
       [ 1.5373707e-03],
       [-6.9855668e-02],
       [-6.7473352e-01],
       [-6.5093118e-01]], dtype=float32)

### Сохранение и востановление моделей

После обучения необходимо сохранить параметры модели на диск, чтобы при необходимости к ней можно было бы вернуться позднее, использовать в другой программе, сравнить с другими моделями и т.д. Кроме того интересно было бы сохранять модель через одинаковые промежутки времени во время обучения, чтобы в случае отказа системы обучение можно было бы продолжить с последней контрольной точки. 

В библиотеке tensorflow сохранение и востановление происходит с помощью узла **Saver**. Просто в конце стадии построения создаем узел **Saver**(после того как созданы все переменные). Затем на стадии построения вызываем метод **save()** для сохранения.

In [43]:
reset_graph() 

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)    
training_op = optimizer.minimize(mse)                                        

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Эпоха', epoch, 'MSE=', mse.eval())
            save_path = saver.save(sess, os.path.join(path, 'tmp/my_model.ckpt'))
            
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess, os.path.join(path, 'tmp/my_model_final.ckpt'))

Эпоха 0 MSE= 9.161543
Эпоха 100 MSE= 0.7145006
Эпоха 200 MSE= 0.56670463
Эпоха 300 MSE= 0.5555716
Эпоха 400 MSE= 0.5488117
Эпоха 500 MSE= 0.5436362
Эпоха 600 MSE= 0.53962916
Эпоха 700 MSE= 0.53650916
Эпоха 800 MSE= 0.5340678
Эпоха 900 MSE= 0.53214717


Для восставновления модели достаточно точно так же создать объект Saver на стадии построения, но затем на стадии выполнения вместо инициализации вызываем метод restore() объекта Saver.

In [44]:
reset_graph()

# Можно также сохранить и востановить весь граф(он также срхраняется автоматически при сохранении полной сессии)
# Это позволяет полностью востановить модель, включая структуру графа и значения переменных,
# бех необходимости в поиске кода, который ее построил.
saver = tf.train.import_meta_graph(os.path.join(path, "tmp\my_model_final.ckpt.meta")) 
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, os.path.join(path, 'tmp\my_model_final.ckpt'))
    best_theta_eval = theta.eval()

INFO:tensorflow:Restoring parameters from C:\Users\Juninho\MLProjects\Study\Aurelien Geron - Hands-on ML with Tensorflow and Scikit-learn\Tensorflow\Tensorflow\Data\tmp\my_model_final.ckpt


In [45]:
np.allclose(best_theta_eval, best_theta)

True

In [46]:
# Для частичного сохранения параметров сессии используем:
saver = tf.train.Saver({'weights': theta}) # Созраняются только веса.

###  Визуализация графа и кривых обучения с помощью TensorBoard

Инструменту TensorBoard можно передать статистические данные по обучению, то он отобразит в браузере интерактивные визуализации. Также ему можно передать определение графа, тогда он предложит великолепный интерфейс для его просмотра.

Для начала нужно скорреткировать программу, так чтобы она записывала определение графа и статистические данные по обучению - к примеру, ошибку MSE - в журнальные каталог, из которого и будет читать TensorBoard. При каждом запуске программы необходимо использовать другой журнальный каталог, иначе TensorBoard будет  объединять статистические данные из разных запусков, что испортит визуализацию. Простейшим решением является включение отметки времени в имя журнального каталога.

In [47]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [48]:

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [49]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [50]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [51]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()   

In [52]:
file_writer.close()

In [53]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

In [54]:
# from tensorflow_graph_in_jupyter import show_graph

In [60]:
# show_graph(tf.get_default_graph())

In [58]:
# import tfgraphviz as tfg

In [61]:
# g = tfg.board(tf.get_default_graph())
# g.view()

### Пространства имен

In [62]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [63]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [64]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [65]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


In [66]:
print(error.op.name)

loss/sub


In [67]:
print(mse.op.name)

loss/mse


In [68]:
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


### Модульность

In [69]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

In [70]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [71]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [72]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [73]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

### Совместное использование переменных

In [74]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [75]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [76]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [77]:
reset_graph()

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

In [78]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [79]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

In [81]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [82]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

In [83]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
file_writer.close()

In [84]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [85]:
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()